# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [18]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pi

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-riudcaj5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-riudcaj5
  Resolved https://github.com/huggingface/transformers.git to commit 7f5077e53682ca855afc826162b204ebf809f1f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936473 sha256=d7a561466dfa9d1ab61b80019e956b7fd59f16c9227746c50e792f902b765b15
  Stored in directory: /tmp/pip-ephem-wheel-cache-deufkzck/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.7 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [5]:
model_name = "defog/sqlcoder-7b" # it’s a model called sqlcoder-7b

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [7]:
bnb_config = BitsAndBytesConfig(    # how the model should be shrunk (quantized)
  load_in_4bit=True,                 # my config to 4 bits
  bnb_4bit_use_double_quant=True,             # adding an extra step to make the shrinking process even more efficient
  bnb_4bit_quant_type="nf4",                 #adding an extra step nf4
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [8]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,   #
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [10]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400): #sqlcoder-7b) | Write an SQL query | max num of words 400 = default
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],  #important words
        num_return_sequences=1,                   #asking the model to generate only one response
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5  #5 different ways to write the response and picking the best one
    )
    return outputs

[link text](https://)# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [11]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

### Input
Question: Find all passengers on flight number SV237.
Database Schema:
CREATE TABLE passengers (
    passenger_id INT PRIMARY KEY,
    name TEXT,
    age INT,
    gender TEXT,
    nationality TEXT
);
CREATE TABLE flights (
    flight_id INT PRIMARY KEY,
    flight_number TEXT,
    departure_airport TEXT,
    arrival_airport TEXT,
    departure_time DATETIME,
    arrival_time DATETIME,
    airline TEXT
);
CREATE TABLE tickets (
    ticket_id INT PRIMARY KEY,
    passenger_id INT,
    flight_id INT,
    seat_number TEXT,
    ticket_price FLOAT,
    FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id),
    FOREIGN KEY (flight_id) REFERENCES flights(flight_id)
);

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [12]:
sp_nl2sql = sp_nl2sql.format(question="Return all passengers on flight SV237")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

### Input
Question: Find all passengers on flight number SV237.
Database Schema:
CREATE TABLE passengers (
    passenger_id INT PRIMARY KEY,
    name TEXT,
    age INT,
    gender TEXT,
    nationality TEXT
);
CREATE TABLE flights (
    flight_id INT PRIMARY KEY,
    flight_number TEXT,
    departure_airport TEXT,
    arrival_airport TEXT,
    departure_time DATETIME,
    arrival_time DATETIME,
    airline TEXT
);
CREATE TABLE tickets (
    ticket_id INT PRIMARY KEY,
    passenger_id INT,
    flight_id INT,
    seat_number TEXT,
    ticket_price FLOAT,
    FOREIGN KEY (pas

In [13]:
# Import the required class
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('defog/sqlcoder-7b')


# Define the get_outputs function
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Tokenize the input prompt
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')

# Generate the SQL query using the model
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)

# Decode the model's response into readable text
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

print(SQL[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

### Input
Question: Find all passengers on flight number SV237.
Database Schema:
CREATE TABLE passengers (
    passenger_id INT PRIMARY KEY,
    name TEXT,
    age INT,
    gender TEXT,
    nationality TEXT
);
CREATE TABLE flights (
    flight_id INT PRIMARY KEY,
    flight_number TEXT,
    departure_airport TEXT,
    arrival_airport TEXT,
    departure_time DATETIME,
    arrival_time DATETIME,
    airline TEXT
);
CREATE TABLE tickets (
    ticket_id INT PRIMARY KEY,
    passenger_id INT,
    flight_id INT,
    seat_number TEXT,
    ticket_price FLOAT,
    FOREIGN KEY (pas

In [14]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [15]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT passengers.name, passengers.age, passengers.gender, passengers.nationality FROM passengers JOIN tickets ON passengers.passenger_id = tickets.passenger_id JOIN flights ON tickets.flight_id = flights.flight_id WHERE flights.flight_number = 'SV237';


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [23]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   YOUR TABLES HERE


  create table flights(
    flight_id INT primary key, -- Unique ID for flight
    flight_number VARCHAR, -- Flight number
    departure_time DATETIME,
    arrival_airport VARCHAR, -- Airport of arrival
    assigned_pilot INT, -- Pilot assigned to the flight
    assigned_crew INT, -- Crew assigned to the flight
    foreign key (assigned_pilot) REFERENCES employees(ID_Usr),
    foreign key (assigned_crew) REFERENCES employees(ID_Usr)
    );

   create table salary(
        ID_Usr INT,-- Unique Id for employee
        year DATE, -- Date
        salary FLOAT, --Salary of employee
        foreign key (ID_Usr) references employees(ID_Usr) -- Join Employees with salary
        );

    ### Response
    YOUR QERIES AND SAMPLE RESPONSES HERE

    Question: `Find all flights departing from Jeddah (JED)`:
    SELECT flight_number, departure_time, arrival_airport
    FROM flights
    WHERE departure_airport = 'JED';

    Question: `How Many employes we have with a salary bigger than 50000?`:
    SELECT SUM(salary) AS total_salary_2025
    FROM salary
    WHERE YEAR(year) = 2025;

    `{question}`:
    ```sql3
    """

In [17]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   YOUR TABLES HERE


  create table aviation(
    flight_id INT PRIMARY KEY, -- Unique ID for flight
    flight_number VARCHAR(50), -- Flight number
    departure_airport VARCHAR(100), -- Departure airport
    arrival_time DATETIME, -- Arrival time
    assigned_pilot INT, -- Pilot assigned to the flight
    assigned_crew INT, -- Crew assigned to the flight
    foreign key (assigned_pilot) REFERENCES employees(ID_Usr), 
    foreign key (assigned_crew) REFERENCES employees(ID_Usr)
    );

  create table m

In [24]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [26]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

1.   List item
2.   List item



In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [35]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

### Tables:
CREATE TABLE flights (
    flight_id INT PRIMARY KEY,
    flight_number VARCHAR(50),
    assigned_pilot INT,
    assigned_crew INT,
    FOREIGN KEY (assigned_pilot) REFERENCES employees(ID_Usr),
    FOREIGN KEY (assigned_crew) REFERENCES employees(ID_Usr)
);

### Samples:
Question: `Find all flights departing from Jeddah (JED)`:
SELECT flight_number, departure_time, arrival_airport
FROM flights
WHERE departure_airport = 'JED'
LIMIT 10;

  #### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [36]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

### Tables:
CREATE TABLE flights (
    flight_id INT PRIMARY KEY,
    flight_number VARCHAR(50),
    assigned_pilot INT,
    assigned_crew INT,
    FOREIGN KEY (assigned_pilot) REFERENCES employees(ID_Usr),
    FOREIGN KEY (assigned_crew) REFERENCES employees(ID_Usr)
);

### Samples:
Question: `Find all flights departing from Jeddah (JED)`:
SELECT flight_number, departure_time, arrival_airport
FROM flights
WHERE departure_airport = 'JED'
LIMIT 10;

  #### Response
    Based on your instructions, here is t

In [37]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [38]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.first_name, employees.last_name, MAX(salary) AS max_salary
     FROM employees
     GROUP BY employees.first_name, employees.last_name
     ORDER BY max_salary DESC
     LIMIT 1;


#Now the question in spanish.


In [39]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Busca todos los vuelos que salen de Jeddah?")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

### Tables:
CREATE TABLE flights (
    flight_id INT PRIMARY KEY,
    flight_number VARCHAR(50),
    assigned_pilot INT,
    assigned_crew INT,
    FOREIGN KEY (assigned_pilot) REFERENCES employees(ID_Usr),
    FOREIGN KEY (assigned_crew) REFERENCES employees(ID_Usr)
);

### Samples:
Question: `Find all flights departing from Jeddah (JED)`:
SELECT flight_number, departure_time, arrival_airport
FROM flights
WHERE departure_airport = 'JED'
LIMIT 10;

  #### Response
    Based on your instructions, here is t

In [33]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [34]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT flight_number, departure_time, arrival_airport FROM flights WHERE departure_airport = 'JED' LIMIT 10;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
For easy questions, a simple prompt works fine. But for harder questions, examples are needed.